In [266]:
from datetime import datetime
class App_Logger:
    def __init__(self,file="logfile.csv"):
        self.f_name =file
    def log(self,log_type,log_msg):
        now = datetime.now()
        current_time = now.strftime("%d-%m-%Y %H:%M:%S")
        f= open(self.f_name, "a+")
        f.write(current_time+","+log_type+","+log_msg+"\n")
        f.close()


In [267]:
import mysql.connector as connection
class sql:
    def __init__(self,host,user,passwd,db=""):
        self.host=host
        self.user=user
        self.passwd=passwd
        self.db=db
        self.logger=App_Logger("logfile.txt")
        self.logger.log("info", "SQL object created")
    def conn(self):
        try:
            if self.db=="":
                return connection.connect(host=self.host,user=self.user,passwd=self.passwd)
            else:
                return connection.connect(host=self.host,user=self.user, database=self.db,passwd=self.passwd)
        except Exception as e:
            self.logger.log("error", f"connection error : {str(e)}")
            print(str(e))
            
    def db_list(self):
        try:
            conn=self.conn()
            cursor = conn.cursor() #create a cursor to execute queries
            q="SHOW DATABASES"
            cursor.execute(q)
            print(cursor.fetchall())
            conn.close()
            self.logger.log("info", "DB list displayed")
        except Exception as e:
            conn.close()
            print(str(e))
            self.logger.log("error", f"db list error : {str(e)}")
    
    def create_db(self,db_name):
        try:
            conn=self.conn()
            cursor = conn.cursor()
            cursor.execute(f"create database {db_name}")
            self.db=db_name
            conn.close()
            self.logger.log("info", f"{db_name} DB created")
        except Exception as e:
            conn.close()
            print(str(e))
            self.logger.log("error", f"db not created error : {str(e)}")
            
    def create_table(self,table_name,columns):
        try:
            conn=self.conn()
            cursor = conn.cursor()
            cursor.execute(f"CREATE TABLE {table_name} ({columns})")
            conn.close()
            self.logger.log("info", f"{table_name} table created with columns: {columns}")
        except Exception as e:
            conn.close()
            print(str(e))
            self.logger.log("error", f"table not created error : {str(e)}")
            
            
    def insert(self,table_name,data):
        try:
            conn=self.conn()
            cursor = conn.cursor()
            #print(f"INSERT INTO {table_name} VALUES ({data})")
            cursor.execute(f"INSERT INTO {table_name} VALUES ({data})")
            conn.commit()
            conn.close()
        except Exception as e:
            conn.close()
            print(str(e))
            self.logger.log("error", f"insert error : {str(e)}")
            
            
    def dump_file(self,f_name,t_name,columns,p_process=False):
        try:
            f=open(f_name,"r")
            f.readline()
            self.create_table(t_name,columns)
            for line in lines:
                if p_process:
                    /
                    data="\'"+preprocess(line[:-1]).replace(",","\',\'")+"\'"
                else:
                    data="\'"+line[:-1].replace(",","\',\'")+"\'"
                self.insert(t_name,data)
            self.logger.log("info", f"{f_name} file data dumped to {t_name} table")
        except Exception as e:
            print(str(e))
            self.logger.log("error", f"file dump error : {str(e)}")
            

    def select_db(self,db_name):
        self.db=db_name
        self.logger.log("info", f"{db_name} DB selected")

    def show(self,t_name):
        try:
            conn=self.conn()
            cursor = conn.cursor()
            cursor.execute(f"Select * from {t_name}")
            for result in cursor.fetchall():
                print(result)
            conn.close()
            self.logger.log("info", f"{t_name} table displayed")
        except Exception as e:
            conn.close()
            print(str(e))
            self.logger.log("error", f"table show error : {str(e)}")
            
        




In [268]:
def preprocess(line):
    return line.replace(";0","").replace(";",',')

In [269]:
ob=sql("localhost","root","2001")


In [270]:
ob.select_db("UCI")

In [271]:
columns="Chiral_indice_n INT(2),Chiral_indice_m INT(2),Initial_atomic_coordinate_u INT(10),Initial_atomic_coordinate_v INT(10),Initial_atomic_coordinate_w INT(10),Calculated_atomic_coordinates_u INT(10),Calculated_atomic_coordinates_v INT(10),Calculated_atomic_coordinates_w INT(10)"


In [272]:
    
ob.dump_file("carbon_nanotubes.csv","test2",columns,True)

1050 (42S01): Table 'test2' already exists
1136 (21S01): Column count doesn't match value count at row 1
1136 (21S01): Column count doesn't match value count at row 1
1136 (21S01): Column count doesn't match value count at row 1
1136 (21S01): Column count doesn't match value count at row 1
1136 (21S01): Column count doesn't match value count at row 1


In [273]:
ob.show("test2")

(2, 1, 679005, 701318, 17033, 721039, 730232, 17014)
(2, 1, 717298, 642129, 231319, 738414, 65675, 232369)
(2, 1, 489336, 303751, 88462, 477676, 263221, 88712)
(2, 1, 413957, 632996, 40843, 408823, 657897, 39796)
(2, 1, 334292, 543401, 15989, 303349, 558807, 157373)
(2, 1, 510664, 696249, 255128, 496977, 725608, 25597)
(2, 1, 413957, 632996, 374176, 390878, 657645, 374735)
(2, 1, 712552, 55479, 112271, 750056, 561494, 114944)
(2, 1, 320995, 298682, 1837, 281301, 261433, 182779)
(2, 1, 394742, 279367, 302747, 35086, 2335, 302196)
(2, 1, 282702, 357871, 397985, 229138, 328729, 398581)
(2, 1, 320995, 298682, 517033, 278752, 269615, 51733)
(2, 1, 605258, 720633, 469414, 613415, 760009, 469452)
(2, 1, 510664, 696249, 588462, 521991, 736991, 588946)
(2, 1, 679005, 701318, 6837, 718811, 738577, 68244)
(2, 1, 605258, 720633, 802747, 649178, 766301, 80193)
(2, 1, 665708, 456599, 326557, 680389, 444562, 327502)
(2, 1, 712552, 55479, 445605, 737216, 552056, 443517)
(2, 1, 586043, 367004, 540843, 